In [1]:
import re
import os

%matplotlib inline

import numpy as np
import pandas as pd
import pylab as P
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.probability import FreqDist

In [2]:
# with open('enron/allen-p/_sent_mail/6', 'r') as content_file:
#     content = content_file.read()
#     print(re.findall('[\n\r].*From: \s*([^\n\r]*)', content)[0])
#     print(re.findall('[\n\r].*To: \s*([^\n\r]*)', content)[0])
#     print(re.findall('Subject: (?s)(.*)Mime-Version', content)[0])
# #     print(re.findall('[\n\r].*Date: \s*([^\n\r]*)', content)[0])
# #     text_original = re.findall("(?<=X-FileName: )[.|\n|\W|\w]*", content)[0]
# #     text = text_original.replace('\n', '')
# #     print(text)

In [3]:
path = 'enron/allen-p/_sent_mail/'

from_list = []
to_list = []
subject_list = []
date_list = []
text_list = []

for file in os.listdir(path):
    with open(os.path.join(path, file), 'r') as content_file:
        content = content_file.read()
        from_list.append(re.findall('[\n\r].*From: \s*([^\n\r]*)', content)[0])
        to_list.append(re.findall('[\n\r].*To: \s*([^\n\r]*)', content)[0])
        subject = re.findall('Subject: (?s)(.*)Mime-Version', content)[0]
        subject = subject.replace('\n', '')
        subject_list.append(subject)
        date_list.append(re.findall('[\n\r].*Date: \s*([^\n\r]*)', content)[0])
        text_original = re.findall("(?<=X-FileName:)[.|\n|\W|\w]*", content)[0]
        text = text_original.replace('\n', '')
        text = text.replace('pallen (Non-Privileged).pst', '')
        text = text.replace('pallen.nsf', '')
        text_list.append(text)

data = {'From': from_list, 'To': to_list, 'Subject': subject_list, 'Date': date_list, 'Text': text_list}

df = pd.DataFrame(data, columns=['From', 'To', 'Subject', 'Date', 'Text'])
df.head(10)      


,From,To,Subject,Date,Text
0,phillip.allen@enron.com,tim.belden@enron.com,,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes th...
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!
3,phillip.allen@enron.com,randall.gay@enron.com,,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy, Can you send me a schedule of the sala..."
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.
5,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)","Greg, How about either next Tuesday or Thursd..."
6,phillip.allen@enron.com,"david.l.johnson@enron.com, john.shafer@enron.com",,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",Please cc the following distribution list wit...
7,phillip.allen@enron.com,joyce.teixeira@enron.com,Re: PRC review - phone calls,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",any morning between 10 and 11:30
8,phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",1. login: pallen pw: ke9davis I don't think ...
9,phillip.allen@enron.com,zimam@enron.com,FW: fixed forward or other Collar floor gas pr...,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",---------------------- Forwarded by Phillip K...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 0 to 601
Data columns (total 5 columns):
From       602 non-null object
To         602 non-null object
Subject    602 non-null object
Date       602 non-null object
Text       602 non-null object
dtypes: object(5)
memory usage: 16.5+ KB


In [5]:
df.describe()

,From,To,Subject,Date,Text
count,602,602,602,602,602
unique,2,206,324,593,580
top,phillip.allen@enron.com,stagecoachmama@hotmail.com,,"Mon, 25 Sep 2000 07:01:00 -0700 (PDT)",Have him send his resume to Karen Buckley in ...
freq,598,40,175,2,2


In [6]:
df_to = df.dropna(subset=['To'])
df['To'].value_counts()[:20]

stagecoachmama@hotmail.com                       40
jsmith@austintx.com                              38
ina.rangel@enron.com                             37
pallen70@hotmail.com                             28
keith.holst@enron.com                            24
cbpres@austin.rr.com                             20
john.lavorato@enron.com                          16
maryrichards7@hotmail.com                        15
jacquestc@aol.com                                14
mike.grigsby@enron.com                           13
llewter@austin.rr.com                            10
rlehmann@yahoo.com                                8
stouchstone@natsource.com                         7
jeff.richter@enron.com                            6
mike.grigsby@enron.com, keith.holst@enron.com     6
mac.d.hargrove@rssmb.com                          6
matthew.lenhart@enron.com                         6
andrea.richards@enron.com                         6
tim.belden@enron.com                              5
al.pollard@e

In [7]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)
organizations = []
contact = []
for email in df['To']:
    if regex.match(email):
        organizations.append(regex.match(email).groups()[1])
        contact.append(regex.match(email).groups()[0])
    else:
        organizations.append('')
        contact.append('')

df['ToOrganization'] = organizations
df['Contact'] = contact
df.head()

,From,To,Subject,Date,Text,ToOrganization,Contact
0,phillip.allen@enron.com,tim.belden@enron.com,,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Here is our forecast,enron,tim.belden
1,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Traveling to have a business meeting takes th...,enron,john.lavorato
2,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",test successful. way to go!!!,enron,leah.arsdall
3,phillip.allen@enron.com,randall.gay@enron.com,,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)","Randy, Can you send me a schedule of the sala...",enron,randall.gay
4,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Let's shoot for Tuesday at 11:45.,enron,greg.piper


In [8]:
df['ToOrganization'].value_counts()[:20]

enron                343
hotmail               87
austintx              38
austin.rr             32
aol                   14
yahoo                 12
natsource              7
keyad                  7
intelligencepress      7
rssmb                  6
creativepanel          4
caprock                4
thedoghousemail        3
webtv                  3
ev1                    2
pdq                    2
spbank                 2
sempratrading          2
thermon                2
juno                   2
Name: ToOrganization, dtype: int64

In [9]:
df['Contact'].value_counts()[:20]

stagecoachmama     40
ina.rangel         38
jsmith             38
mike.grigsby       30
pallen70           28
keith.holst        28
cbpres             22
john.lavorato      16
maryrichards7      15
jacquestc          14
matthew.lenhart    12
llewter            11
jeff.richter        9
rlehmann            8
stouchstone         7
gthorse             6
mac.d.hargrove      6
tim.belden          6
andrea.richards     6
tara.sweitzer       5
Name: Contact, dtype: int64

In [10]:
# Email text analysis
words = nltk.tokenize.word_tokenize('Randy, Can you send me a schedule of the sala')
fdist = FreqDist(words)
print(fdist.keys())

dict_keys(['Can', 'sala', 'schedule', 'me', 'of', ',', 'a', 'you', 'the', 'send', 'Randy'])
